# Neurocombat TOP and StrokeMRI harmonized datasets

Note this must be run in the `neuron` or `neuron_go` environment

## import libraries

In [ ]:
import os
import sys

import pandas as pd
import numpy as np

from neuroCombat import neuroCombat

sys.path.insert(0, '../../') # path to functions

import cvasl.harmony as har

## Let's see how we would apply this to our data
We will have to flip it on it's side to make it work

In [ ]:
# Datasets for this work

MRI_path = '../our_datasets/StrokeMRI/'
TOP_path = '../our_datasets/TOP/'
file_name = 'TrainingDataComplete.csv'

TOP_file = os.path.join(TOP_path, file_name)
MRI_file = os.path.join(MRI_path, file_name)

TOP = pd.read_csv(TOP_file, index_col=0)
MRI = pd.read_csv(MRI_file, index_col=0)

In [ ]:
TOP = TOP.drop(['Site', 'ID'], axis= 1)
TOP.head(3)

In [ ]:
MRI = MRI.drop(['Site', 'ID'],axis = 1)
MRI.tail(5)

In [ ]:
both_togetherF, ftF, btF, feature_dictF, len1, len2 = har.prep_for_neurocombat(MRI, TOP)

In [ ]:
both_togetherF

In [ ]:
ftF.to_csv('ftF_top_mri.csv')

In [ ]:
data = np.genfromtxt('ftF_top_mri.csv', delimiter=",", skip_header=1)
data = data[:, 1:]
data

In [ ]:
TOP.head(3)

In [ ]:
both_togetherF.head(3)

In [ ]:
# Specifying the batch (scanner variable) as well as a sex covariate to preserve:
first_columns_as_one = [1] * len1
last_columns_as_two = [2] * len2
covars = {'batch':first_columns_as_one + last_columns_as_two,
          #'Site': both_togetherF.loc['Site',:].values.tolist(), 
          'Sex':both_togetherF.loc['Sex',:].values.tolist(),
           'Age':both_togetherF.loc['Age',:].values.tolist(),} 
covars = pd.DataFrame(covars) 

In [ ]:
# specifify sex as categorical
categorical_cols = ['Sex']
# To specify the name of the variable that encodes for the scanner/batch covariate:
batch_col_mine = 'batch'
our_continuous_col=['Age']
#Harmonization step:
data_combat = neuroCombat(dat=data,
    covars=covars,
    batch_col=batch_col_mine,
    continuous_cols=our_continuous_col,
    categorical_cols=categorical_cols)["data"]

# (n_dat, covars, batch_col, continuous_cols=continuous_cols,
#                                     categorical_cols=categorical_cols)

In [ ]:
neurocombat = pd.DataFrame(data_combat)
neurocombat.head(3)

In [ ]:
topperF = har.make_topper(btF,'Age', 'Sex')

In [ ]:
bottom = neurocombat.reset_index(drop=False)
bottom = bottom.rename(columns={"index": "char"})
bottom.columns = topperF.columns

In [ ]:
back_together = pd.concat([topperF, bottom])
back_together = back_together.T
#back_together

In [ ]:
new_header = back_together.iloc[0] #grab the first row for the header
back_together.columns = new_header #set the header row as the df header
back_together = back_together[1:]
#back_together

In [ ]:
back_together.head(3)

In [ ]:
neuro_harm_top =back_together.tail(len2)
neuro_harm_mri =back_together.head(len1)

In [ ]:
neuro_harm_top = neuro_harm_top.rename(feature_dictF, axis='columns')
neuro_harm_mri = neuro_harm_mri.rename(feature_dictF, axis='columns')

In [ ]:
neuro_harm_mri = neuro_harm_mri.reset_index()
neuro_harm_mri = neuro_harm_mri.rename(columns={"index": "participant_id"})
neuro_harm_mri

Save off to harmonized csv files

In [ ]:
neuro_harm_top = neuro_harm_top.reset_index()
neuro_harm_top = neuro_harm_top.rename(columns={"index": "participant_id"})
neuro_harm_top

In [ ]:
neuro_harm_mri.to_csv('harm_results/neuro_harm_mri_for_1.csv')
neuro_harm_top.to_csv('harm_results/neuro_harm_top_for_1.csv')

In [ ]:
neuro_harm_mri.columns

In [ ]:
number_columns = ['Age', 'Sex', 'GM_vol', 'WM_vol', 'CSF_vol',
       'GM_ICVRatio', 'GMWM_ICVRatio', 'WMHvol_WMvol', 'WMH_count',
       'ACA_B_CoV', 'MCA_B_CoV', 'PCA_B_CoV', 'TotalGM_B_CoV',
       'ACA_B_CBF', 'MCA_B_CBF', 'PCA_B_CBF', 'TotalGM_B_CBF',]

# Data quality check

In [ ]:
#TOP

In [ ]:
#neuro_harm_top

In [ ]:
neuro_harm_mri.isna().sum().sum()

In [ ]:
neuro_harm_top.isna().sum().sum()

In [ ]:
#neuro_harm_top.head(3)
(neuro_harm_mri[number_columns] < 0).sum()

In [ ]:
(neuro_harm_top[number_columns] < 0).sum()

## So this new variable (WMHvol_WMvol ) is again problematic

 We will wait about logging it until we see other columns we may want to log across all datasets